In [ ]:
import os
import requests
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from dotenv import load_dotenv

load_dotenv()

# トークンを取得する関数
def get_token():
    auth_url = os.getenv('AICORE_AUTH_URL')
    client_id = os.getenv('AICORE_CLIENT_ID')
    client_secret = os.getenv('AICORE_CLIENT_SECRET')

    token_url = f"{auth_url}/oauth/token"
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

    response = requests.post(token_url, data=data)
    response.raise_for_status()

    token = response.json().get('access_token')
    if not token:
        raise ValueError("トークンが取得できませんでした。")

    return token

# トークンを取得
token = get_token()
print(f'Token: {token}')

# プロキシクライアントの取得
proxy_client = get_proxy_client('gen-ai-hub', token=token)

# デプロイメント一覧を取得
deployments_list = proxy_client.deployments
print("model_name, deployment_id, config_name")
for deployment in deployments_list:
    print(f"{deployment.model_name}, {deployment.deployment_id}, {deployment.config_name}")

In [ ]:
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings

# 取得したデプロイメントIDを使用
deployment_id = "d8b1e5eb7341b1f3"

# embedding_modelの設定
embedding_model = OpenAIEmbeddings(deployment_id=deployment_id, proxy_client=proxy_client)

In [ ]:
import os
import json
from hdbcli import dbapi
from langchain_community.document_loaders import JSONLoader
from langchain.vectorstores import HanaDB
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
from tqdm import tqdm
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

# ログの設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 環境変数の読み込み
load_dotenv()

# 埋め込みモデルの設定
embedding_model = OpenAIEmbeddings(deployment_id=os.getenv('EMBEDDING_DEPLOYMENT_ID'))

def get_embedding(input_text):
    return embedding_model.embed_query(input_text)

# データベースの接続と初期化
def connect_database():
    try:
        connection = dbapi.connect(
            address=os.getenv("HANA_DB_ADDRESS"),
            port=int(os.getenv("HANA_DB_PORT")),  # ポートは整数に変換
            user=os.getenv("HANA_DB_USER"),
            password=os.getenv("HANA_DB_PASSWORD"),
            autocommit=True,
            sslValidateCertificate=False,
        )
        db = HanaDB(
            connection=connection,
            embedding=embedding_model,
            table_name="FILE_EMBEDDINGS",
        )
        db.delete(filter={})  # データベースの初期化
        logging.info("データベースに接続し、初期化しました。")
        return connection, db
    except Exception as e:
        logging.error(f"データベース接続エラー: {e}")
        raise

# JSONファイルを処理してDBに格納
def process_file(file_path, db, text_splitter, chunk_file_name):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            metadata = data.get('metadata', {})

        loader = JSONLoader(file_path, jq_schema='.content', text_content=False)
        documents = loader.load()
        for doc in documents:
            if doc.page_content:  # 空でないことを確認
                if os.path.basename(file_path) == chunk_file_name:
                    chunks = text_splitter.split_text(doc.page_content)
                    for i, chunk in tqdm(enumerate(chunks)):
                        chunk_doc = doc.copy()  # 元の文書オブジェクトをコピー
                        chunk_doc.page_content = chunk
                        embedding = get_embedding(chunk)
                        chunk_doc.metadata['embedding'] = embedding
                        chunk_doc.metadata['source_filename'] = metadata['filename']
                        chunk_doc.metadata['chunk_id'] = f"{metadata['filename']}_chunk_{i+1}"
                        db.add_documents([chunk_doc])
                else:
                    doc.metadata['embedding'] = get_embedding(doc.page_content)
                    doc.metadata['source_filename'] = metadata['filename']
                    db.add_documents([doc])

                logging.info(f"Processed and added JSON file: {file_path}")
            else:
                logging.warning(f"Skipping empty document: {file_path}")
    except Exception as e:
        logging.error(f"Error processing file {file_path}: {e}")

def process_json_files(json_folder, db, chunk_file_name):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)  # チャンクサイズとオーバーラップを設定
    json_files = [os.path.join(json_folder, file) for file in os.listdir(json_folder) if not os.path.isdir(os.path.join(json_folder, file))]

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(process_file, file_path, db, text_splitter, chunk_file_name) for file_path in json_files]
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing JSON files"):
            future.result()  # 例外をキャッチするため

# メインフロー
def main():
    json_folder = "../data/all_JSONs"
    chunk_file_name = "SAPAICORE.json"  # チャンクしたいファイル名を指定
    try:
        connection, db = connect_database()
        process_json_files(json_folder, db, chunk_file_name)
    except Exception as e:
        logging.error(f"エラーが発生しました: {e}")
    finally:
        connection.close()
        logging.info("データベース接続を閉じました。")

# 実行部分
if __name__ == "__main__":
    main()